In [40]:
from keras.layers import *
from keras import Model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr
import keras.losses as losses
 
 
# Setup the network parameters:
intermediate_dim = 300
latent_dim = 10
beta = 5
epochs = 15
n_sequences = 101
n_features = 7

boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
# boat_csv = boat_csv[:5739]
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_curved = pd.read_csv("Data/Anomalous_boat_data.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0", "heading"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved)
print(anomalous_data.shape)

batch_size =  656
interval = 656
def prepare_sequences(data):
    samples = []
    for i in range(0,data.shape[0]- batch_size, interval):
        sample = data[i:i+batch_size]	
        samples.append(sample)

    sequences = np.array(samples)

    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, 7))

    return trainX

trainX_nominal = prepare_sequences(normal_data) 
print(trainX_nominal.shape)

input_length = trainX_nominal.shape[0]
trainX_anomalous = prepare_sequences(anomalous_data)
print(trainX_anomalous.shape)  


(66256, 7)
(6620, 7)
(100, 656, 7)
(10, 656, 7)


In [43]:
# n_features = 7
# pool_size = 2
# x = Input(shape=(656, 7), name="input")
# h = x
# h = Conv1D(filters=300, kernel_size=n_features,
#            padding='same', name='Conv1')(h)
# h = MaxPooling1D(pool_size=pool_size, name='Maxpool1')(h)
# h = Conv1D(filters=200, kernel_size=n_features,
#            padding='same', name='Conv2')(h)
# h = MaxPooling1D(pool_size=pool_size,  name="Maxpool2")(h)
# h = Conv1D(filters=150, kernel_size=n_features,
#            name='Conv3')(h)
# h = MaxPool1D(pool_size=pool_size, name="Maxpool3")(h)
# h = Conv1D(filters=100, kernel_size=n_features,
#            padding="same", name='Conv4')(h)
# h = MaxPool1D(pool_size=pool_size, name="Maxpool4")(h)
# h = Conv1D(filters=latent_dim, kernel_size=n_features,
#            padding="same", name='Conv5')(h)
# h = MaxPool1D(pool_size=pool_size, name="Maxpool5")(h)
# 
# h = Flatten()(h)
# 
# h = Dense(latent_dim, name='embedding') (h)
# y = h
# 
# y = Dense(656, activation="relu") (y)
# y = Reshape((41, 16))(y)
# y = Conv1D(filters=7, kernel_size=n_features,
#            padding='same', name='conv-decode1')(y)
# y = UpSampling1D(size=2, name='upsampling1')(y)
# y = Conv1D(filters=150, kernel_size=n_features,
#            padding='same', name='conv-decode2')(y)
# y = UpSampling1D(size=2, name='upsampling2')(y)
# y = Conv1D(filters=200, kernel_size=n_features,
#            padding='same', name='conv-decode3')(y)
# y = UpSampling1D(size=2, name='upsampling3')(y)
# y = Conv1D(filters=7, kernel_size=n_features,
#            padding='same', name='conv-decode4')(y)
# y = UpSampling1D(size=2, name='upsampling4')(y)
# # y = Conv1D(filters=7, kernel_size=n_features,
# #            padding='same', name='conv-decode5')(y)
# # y = UpSampling1D(size=2, name='upsampling5')(y)
# 
# 
# model = Model(inputs=x, outputs=y, name='AE')
# enc = Model(inputs=x, outputs=h, name='encoder')
# 
# model.summary()


model = Sequential()

encoder = Sequential()
encoder.add(Conv1D(filters=200, input_shape=(None,7), padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=150, padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=100,padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=50, padding='same',
                   kernel_size=n_features))
encoder.add(Conv1D(filters=latent_dim, padding='same',
                   kernel_size=n_features))

decoder = Sequential()
decoder.add(Dense(units=n_features))

model.add(encoder)
model.add(decoder)

model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

model.fit(x=boat_curved, y=boat_curved, epochs=epochs)




ValueError: Error when checking input: expected sequential_2_input to have 3 dimensions, but got array with shape (6620, 7)

In [200]:
from sklearn.manifold import TSNE

def tsne(data, title):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(data)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"],y=tsne_df["Y"], alpha=0.5)
    plt.title(title)
    plt.show()
    
    return tsne_df


In [209]:
weight_modifiers = np.ones((latent_dim,))

nom_enc =(encoder.predict(trainX_nominal))
print(nom_enc.shape)

nom_enc = nom_enc.reshape(100,latent_dim)

# weight_modifiers[0] = 1
# nom_enc_modified = nom_enc
# for i, elem in enumerate(nom_enc_modified):
#     nom_enc_modified[i] = elem * weight_modifiers    
for i,elem in enumerate(nom_enc):
    plt.scatter(y=nom_enc[i], x=np.linspace(1,latent_dim,latent_dim))
plt.show()


# tsne_enc_df = tsne(np.average(nom_enc, axis=0), "Encoded Nominal")
# 
# 
# nom_dec = decoder.predict(nom_enc)
# tsne_dec_df = tsne(np.average(nom_dec, axis=0), "Decoded Nominal")


(100, 10)


ValueError: cannot reshape array of size 1000 into shape (100,40)

In [91]:
df_nominal = pd.DataFrame(normal_data, columns=boat_csv.columns)
df = pd.DataFrame(np.average(nom_enc, axis=0))

plt.figure(1)
axis_list = []

for i in range(df.shape[1]):
    plt.plot(df[i])

plt.show()    

In [23]:
from sklearn.cluster import KMeans
cluster_comp = [0,1,2,3]# print(tsne_enc_df)
# print(tsne_dec_df)

kmeans_normal = KMeans(n_clusters=4, random_state=0).fit(normal_data)
clusters_normal = kmeans_normal.predict(np.average(trainX_nominal,axis=0))

kmeans_enc = KMeans(n_clusters=4, random_state=0).fit(tsne_enc_df)
clusters_enc = kmeans_enc.predict(tsne_enc_df)

kmeans_dec = KMeans(n_clusters=4, random_state=0).fit(tsne_dec_df)
clusters_dec = kmeans_dec.predict(tsne_dec_df)

In [24]:
def transform_to_mask(cl_label, clust_obj):
    mask = []
    part = []
    for i, elem in enumerate(clust_obj):
        if elem == cl_label:
            part.append(i)
        else:
            if part:
                mask.append(part)
                part = []
            else:
                pass
    return mask


def get_mask_list(clust_obj):
    mask_list = []
    for i in cluster_comp:
        mask_list.append(transform_to_mask(i, clust_obj))
    return mask_list


masks_normal = np.array(get_mask_list(clusters_normal))

masks_enc = np.array(get_mask_list(clusters_enc))

masks_dec = np.array(get_mask_list(clusters_dec))

masks = (masks_normal, masks_enc,masks_dec)
print(masks_normal.shape, masks_enc.shape, masks_enc.shape)


(4,) (4,) (4,)


In [73]:
def plot_cl(cl, color):
    plt.plot(cl["Lon"], cl["Lat"], color=color)
        
    
def plot_clusters_on_map():
    titles = ['Nominal Normal','Nominal Encoded','Nominal Decoded']
    color_list = ['blue','green','red','black']
    map = normal_data
    print(map.shape)
    for k,mask in enumerate(masks):
        for i, elem in enumerate(mask):
            for j in elem:
                cl = pd.DataFrame(map[j], columns=boat_csv.columns)
                plot_cl(cl, color_list[i]) 
        plt.title(titles[k])
        plt.show()
      
               
plot_clusters_on_map()


(66963, 7)
